# Recipes PCA
### By Brian Kitano

Okay, I'm going to use the Epicurious dataset to identify palates common across their recipes. 

## Introduction / Hypothesis

## Materials

## Procedure
1. Download the JSON data
2. Parse the JSON to extract recipe names and ingredients with their quantities.
3. Create the data matrix M, where each column is a recipe and each row is an ingredient; the entry is the quantity in a normalized and standardized quantity (grams?)
4. PCA

Bonus: construct the bipartite graph of ingredients to recipes, and then project it down onto a unipartite graph of ingredients where the weight of each edge is the frequency of connections. 

In [2]:
# 1. Parse the ingredients to extract recipe names and ingredients with their quantities
import json

# load in the epicurious set
with open('full_format_recipes.json') as f:
    data = json.load(f)
    f.close()
    
# print data[0]['ingredients']
# len(data) = 20130

# need to filter out the recipes that don't have ingredients listed
# extractIngredients = lambda i: data[i].keys


In [3]:
def containsIngredients(index):
    if 'ingredients' in data[i].keys():
        return i
    else:
        return 0

# get all of the indices which contain ingredients
cleanedIndices = [ containsIngredients(i) for i in range(len(data)) ]

# get all of the ingredient lists as a list of lists
cleanedIngredientsLists = [ data[i]['ingredients'] for i in cleanedIndices ]

# flatten this list, which might contain duplicates
cleanedIngredients = [ ingredient for ingredientList in cleanedIngredientsLists for ingredient in ingredientList]
print len(cleanedIngredients)

# remove any duplicates
uniqueCleanedIngredients = list(set(cleanedIngredients))
print len(uniqueCleanedIngredients)

199315
83465


### Data Cleaning
Before we write all of the ingredients to a file, we should do some NLP cleaning. In looking at the results of the first model run, it seems like to be conservative we should remove all the text that occurs in parentheses, as this seems to really mess up the CRF's ability to identify units.

#### remove things in parentheses (use regex)

In [4]:
import re

# remove all the text that is inside a parenthesis
noParenthesisIngredients = [re.sub(r'\([^)]*\)', '', ingredient) for ingredient in uniqueCleanedIngredients]

print len(noParenthesisIngredients)

83465


#### dealing with the word "plus"

More complex problem. There are lots of ways that "plus" is used. Some examples:

##### when quantities don't add nicely
- "1/2 cup plus 1 1/2 tablespoons red wine vinegar"
- "1/2 cup plus 2 tablespoons granola"
- "1/4 cup plus 1 tablespoon warm water"
- "1/4 teaspoon plus 1/3 cup sugar"
- "2 tablespoons plus 1/2 cup chopped fresh dill"
- "1 tablespoon plus 1/2 teaspoon Dijon mustard"
- "2/3 cup plus 6 tablespoons coarsely chopped pecans"
- "1 cup plus 2 tablespoons whole milk"
- "1 1/2 cups plus 2 tablespoons sugar"
- "1 1/2 cups plus 2 tablespoons water"
- "1 tablespoon plus 3/4 teaspoon ground cinnamon"
- "1 tablespoon plus one teaspoon fresh lemon juice"

These are in a consistent format of UNIT QUANTITY PLUS UNIT QUANTITY INGREDIENT. If we add PLUS as a label, then over the 3k samples we have we might improve, but we might also tag some things as being PLUS when we don't want them to be.

##### when there's a suggestion for more on the side (not a lot of errors there)
- "1/4 cup olive oil, plus more for grilling"
- "5 teaspoons all-purpose flour plus more for dusting"
- "2 tablespoons drained capers plus more for serving"
- "1/2 cup freshly grated Parmesan cheese plus additional for passing"
- "12 rice-paper rounds, plus more in case some tear"
- "1 can whole tomatoes, plus juice"
- "1 tablespoon chile oil containing sesame oil plus some of sediment from jar"

These ones seem like i can just remove all the words after the plus.

##### other, stupid ones
- "8 cornichons, finely chopped, plus 2 pickled onions from the jar, minced"
- "1/2 cup oil-packed sun-dried tomatoes, chopped, plus 2 tablespoons tomato oil"
- "1 tablespoon fresh rosemary leaves or 1 teaspoon crumbled dried, plus rosemary sprigs for garnish"
- "Juice of 1/4 lime, plus 1 lime wedge for garnish"
- "1 1/2 cups sugar, plus 1/4 cup mixed with 1 tablespoon cinnamon, on a plate"
- "1/2 fennel bulb, finely chopped, plus 1 tablespoon finely chopped fronds"
- "1/4 cup chopped fresh cilantro plus 32 whole fresh cilantro leaves"
- "6 large celery stalks, thickly sliced, plus 2 1/2 cups 1/2-inch-thick slices"
- "6 fresh mint leaves plus 1 mint sprig for garnish"

So also there's like a utility function that might need to be taken into account: we really want our data to fit the format nicely of having a name, a unit, and a quantity. 

A really, really easy way to deal with all of this is just to get rid of all the "plus" ingredient listings, which are only ~3000 out of the 83k samples. It might mess up the data but it's easier. Also none of this is training or testing data, this is like actually "I need this" data so it's convenient if I just scrap the shitty stuff. It will also probably have come up in other sections. 

In [5]:
# we use a regex to tag an igredient any time "plus" appears as a word with or without a comma on its own
noPlusListings = list(filter(lambda ingredient: (re.search("\s*(plus)\,*\s*", ingredient) == None), noParenthesisIngredients))

len(noPlusListings)

80480

#### dashes, commas, and other grammar thingies
might be worth removing all of that, but not going to yet. 

##### Asterisks (*)
Asterisks appear in two variants:
"2 1/2 pounds Jerusalem artichokes *" where the asterisk is at the end, and "*seedless red grapes" where it's indicating that this is the start of a comment. We can thus remove anything after an asterisk, since it doesn't matter in either case.

In [6]:
# removing all the text after an asterisk
asteriskFreeListings = [ re.sub("\*.*\n*",'',ingredient) for ingredient in noPlusListings ]

#### "a" and "an"
This probably maps to the number 1 right?

#### typos

like fam what how is that even ugh how do i check for typos here. 

#### "or"
we could remove all the tokens after the word "or", since it's optional.

In [7]:
# remove all the things after an or
noOrListings = [ re.sub("[^A-z]\.*\,*\s*(or|OR|Or)+.*",'', ingredient) for ingredient in asteriskFreeListings ]

Okay, now let's write this clean stuff to a file.

In [8]:
# write the ingredients to a file, which we'll then feed to a model
with open('ingredientsList.txt', 'a') as the_file:
    for ingredient in noOrListings:
        if ingredient != "":
            asciiOnlyIngredient = "".join(i for i in ingredient if ord(i)<128)
            ingredientString = asciiOnlyIngredient + "\n"
            the_file.write(ingredientString)


In [9]:
# okay, the model ran and i've got the sauce

# load in the labeled stuff
with open('results.json') as g:
    labeledIngredients = json.load(g)
    g.close()
    
print labeledIngredients[0]['name']
print labeledIngredients[0]['unit']
print labeledIngredients[0]['qty']

lemon juice
cup
1 1/4


In [14]:
# now we need to normalize all of the units and measures. We'll use milliliters for volume and grams for mass.

# first we'll get a list of all the units
def containsUnit(i):
    if 'unit' in labeledIngredients[i].keys():
        return i
    else:
        return 0
    
# get all of the indices which contain units
unitContainingIndices = [containsUnit(i) for i in range(len(labeledIngredients))]
unitContainingIndices = list(set(unitContainingIndices))

# get all of the units
unitList = [labeledIngredients[i]['unit'] for i in list(set(unitContainingIndices))]

# remove duplicates
uniqueUnitList = list(set(unitList))

print len(uniqueUnitList) # 764 bruh

92


### Pre and Post Modeling Cleaning
What cleaning should be done before we feed the model, and what cleaning should be done after? Also, should we change our factor functions? 

Well, let's think quantitatively about what cleaning means now. We've identified the units from the model, and they're obviously not perfect. We should look at whether we can just cut the shitty ones out now.

In [19]:
# oh man some ingredients don't have a unit floof
# we somehow need to deal with that so

sortedIngredientsByUnit = dict()

for ingredient in labeledIngredients:
    # if there's a unit associated with the ingredient
    if 'unit' in ingredient.keys():
        unit = ingredient['unit']
    else:
        unit = 'na'
        
    # if that unit is already in the dictionary
    if unit in sortedIngredientsByUnit.keys():
        sortedIngredientsByUnit[unit].append(ingredient)
    else:
        # that unit is unseen, so we need to create it
        sortedIngredientsByUnit[unit] = list(ingredient)

unitQuantities = dict()
for unit in sortedIngredientsByUnit.keys():
    unitQuantities[unit] = len(sortedIngredientsByUnit[unit])

for key, value in sorted(unitQuantities.iteritems(), key=lambda (k,v): (v,k)):
    print "%s: %s" % (key, value)

12-ounce bottle: 5
branch: 5
fifth: 5
pair: 5
pinch sprig: 5
stem: 5
12-ounce bag: 6
12-ounce bunch: 6
cup piece: 6
cup stalk: 6
cup strip: 6
cup tablespoon cup: 6
link: 6
ounce bunch: 6
ounce can: 6
ounce cup: 6
packet: 6
piece fillet: 6
pound ounce: 6
pound slice: 6
pound stalk: 6
segment: 6
tablespoon stalk: 6
tablespoon teaspoon: 6
bulb tablespoon: 7
bunch sprig: 7
clove clove: 7
cup cup: 7
cup teaspoon: 7
head clove: 7
ounce fillet: 7
pound cup: 7
pound pound: 7
tablespoon cup: 7
tablespoon sprig: 7
can fillet: 8
cup slice: 8
drop: 8
tablespoon tablespoon: 8
teaspoon teaspoon: 8
twist: 8
12-ounce: 9
chunk: 9
log: 9
splash: 9
liter: 10
ball: 11
cup tablespoon: 11
ounce slice: 12
pound fillet: 12
square: 12
wedge: 12
knob: 13
rack: 13
batch: 14
clove teaspoon: 15
gallon: 15
cube: 17
box: 23
cup sprig: 27
sheet: 28
envelope: 29
bulb: 30
dozen: 36
gram: 41
loaf: 45
handful: 54
bag: 57
jar: 58
dash: 81
ear: 111
bottle: 117
strip: 135
fillet: 152
pinch: 160
quart: 172
pint: 242
package: